# 2024/06/05 LODES User Manual
_Author: Meaghan Freund_

### **Introduction**
LODES (LEHD Origin-Destination Employment Statistics) is a collection of state-organized data files, owned by the U.S. Census Bureau and is a part of the LEHD (Longitudinal Employer-Household Data) program. The datasets are collected through LEHD Census data, which is made through combining previously collected survey and administrative data from participating states on jobs, businesses, and workers. 

### **Data Collection and Updates**

LODES data is collected and mapped at the smallest geographic unit that the Census Bureau collects: blocks. This data is collected annually and updated regularly through new versions of LODES. The most recent version, LODES8, spans from 2002-2021 inclusively and maps all blocks with 2020 Census delineations, including blocks that were originally defined in past Census delineations, such as from the 2010 Census.

### **Geographic Coverage**

LODES covers 51 states in the U.S., including defining Washington D.C. as its own state, and territorial partners such as Puerto Rico and the Virgin Islands. However, in LODES8, Puerto Rico is the only territorial partner that is covered. Similarly, certain states do not have files in specific years that LODES provides. Some of the reasons can include a lack of availability of data for a state or a location not being in regular production for LODES.
The current list of exceptions includes:
- *2002*: Arkansas, Arizona, DC, Massachusetts, Mississippi, New Hampshire
- *2003*: Arizona, DC, Massachusetts, Mississippi
- *2004-2009*: DC, Massachusetts
- *2010*: Massachusetts
- *2017-2018*: Alaska
- *2019-2021*: Alaska, Arkansas, Mississippi

These intersections do lack information concerning other files within LODES, but for the ADRIO maker, these exceptions are omitted from the calls altogether.

### **Data Files**

Of the three files available in LODES (RAC, WAC, and OD) the ADRIO maker relies on the Origin-Destination files to map worker flows between residence and workplace locations through matrices. The Origin-Destination files provide the number of individuals who live in a specific home GEOID and commute to a corresponding work GEOID.

### **Additional Resources**

For more detailed information:
- Visit the [LEHD Data Page](https://lehd.ces.census.gov/data/#lodes) on the Census Bureau's website concerning the LODES files.
- Refer to the documentation for the most recent version, [LODES8.1](https://lehd.ces.census.gov/data/lodes/LODES8/LODESTechDoc8.1.pdf).

### **Basic Queries**

The 'label' and 'commuters' are two simple yet imperative queries that show the basic functionality of the LODES ADRIO maker. The 'label' query represents the GEOIDs that are involved with the commuter matrices. The input given by the user for the scope, in this case being states, is translated into a list of GEOIDs. The 'commuters' query shows the total number of workers moving from a home GEOID to a work GEOID as a matrix. The matrix is read so that the rows represent the residence GEOID and the columns are the work location GEOID.

In [1]:
from epymorph.geo.adrio import adrio_maker_library
from epymorph.data_shape import Shapes
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.spec import DynamicGeoSpec, Year, attrib
from epymorph.geography.us_census import (StateScope)

spec = DynamicGeoSpec(
    attributes=[
        attrib('label', str, Shapes.N),
        attrib('commuters', int, Shapes.NxN),
    ],
    time_period=Year(2015),
    scope=StateScope.in_states_by_code(["AZ", "CO", "NV", "NM"]),
    source={
        'label': 'LODES:geoid',
        'commuters': 'LODES'
    }
)

geo = DynamicGeo.from_library(spec, adrio_maker_library)


print(f"Home/Work GEOIDs: {geo['label']}\n")

print(f"Commuters Matrix:\n {geo['commuters']}")

Home/Work GEOIDs: ['04' '08' '32' '35']

Commuters Matrix:
 [[2550132    2582   13263    8100]
 [   1202 2405258     382    5557]
 [   3552     535 1179411     361]
 [   6813    4824     409  764244]]


# Attributes

The 'commuters' attribute outputs the total number of workers commuting, but LODES provides three categories for attributes specifying the type of workers: Age, Monthly Income, and Industry Sectors. Within each category, there are three ranges within them, and the sum of the ranges equals the total number of workers. All of these categories and the total commuters are displayed as NxN matrices of integers, excluding the label query.

## Age
- 'commuters_29_under'
    -  Commuters that are ages 29 and under.
- 'commuters_30_to_54
    -  Commuters that are between the ages of 30 and 54.
- 'commuters_55_over'
    -  Commuters that are ages 55 and over.

## Monthly Income
- 'commuters_1250_under_earnings'
    -  Commuters that earn $1250 and under per month.
- 'commuters_1251_to_3333_earnings'
    -  Commuters that earn between $1251 and $3333 per month.
- 'commuters_3333_over_earnings'
    -  Commuters that earn over $3333 per month.

## Industry Sector
- 'commuters_goods_producing_industry'
    -  Commuters that work in Goods Producing industry sectors.
- 'commuters_trade_transport_utility_industry'
    -  Commuters that work in Trade, Transportation, and Utility industry sectors.
- 'commuters_other_industry'
    -  Commuters that work under all other service industry sectors other than the above claimed industries.


## Job Type
Along with the above categories, LODES provides files detailing different job types and the total number of jobs under that type. However, unlike the attributes, these matrices do not sum to be the total number of workers. 

- 'all_jobs'
    -  All jobs regardless of job type. Allows for multiple jobs per person and is the default when calling the above attributes.
- 'primary_jobs'
    -  Primary jobs, which a primary job is the highest paying job for an individual worker for the year. Limits to one job per worker.
- 'all_private_jobs'
    -  All private jobs, which are privately owned businesses and organizations excluding federal government jobs.
- 'private_primary_jobs'
    -  Primary jobs within the private sector.
- 'all_federal_jobs'
    -  All jobs within the federal government sector.
- 'federal_primary_jobs
    -  Jobs under the federal government sector that are defined as primary jobs.

Below is an example of calling the three different age ranges provided by LODES in a geo spec. The example here loads four counties into the matrices rather than the four states that was used previously.

In [1]:
from epymorph.data_shape import Shapes
from epymorph.geo.spec import DynamicGeoSpec, Year, attrib
from epymorph.geography.us_census import (CountyScope)
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo

spec = DynamicGeoSpec(
    attributes=[
        attrib('label', str, Shapes.N),
        attrib('commuters_29_under', int, Shapes.NxN),
        attrib('commuters_30_to_54', int, Shapes.NxN),
        attrib('commuters_55_over', int, Shapes.NxN),
    ],
    time_period=Year(2015),
    scope=CountyScope.in_counties(["04013", "08041", "32003", "35001"]),
    source={
        'label': 'LODES:geoid',
        'commuters_29_under': 'LODES',
        'commuters_30_to_54': 'LODES',
        'commuters_55_over': 'LODES',
    }
)

geo = DynamicGeo.from_library(spec, adrio_maker_library)

print(f"Commuters ages 29 and under:\n {geo['commuters_29_under']}\n")

print(f"Commuters between ages 30 and 54:\n {geo['commuters_30_to_54']}\n")

print(f"Commuters ages 55 and over:\n {geo['commuters_55_over']}\n")

Commuters ages 29 and under:
 [[369848     11    287    195]
 [    12  43020      9     29]
 [   245     11 186798     48]
 [   119     26     41  55536]]

Commuters between ages 30 and 54:
 [[922520     20    817    354]
 [    19 104126     27     37]
 [   656      7 493523     72]
 [   230     38    120 135000]]

Commuters ages 55 and over:
 [[337141     10    324    151]
 [    18  42355      9     16]
 [   325      3 182295     53]
 [    59     13     56  56352]]

